Working with spatio-temporal GPs in TemporalGPs.jl mostly looks like working with any
other GP. The main differences are to do with what you're allowed to specify as a kernel,
and what kinds of inputs work.

In [1]:
using AbstractGPs
using TemporalGPs

Load up the separable kernel from TemporalGPs. You need to use this to tell TemporalGPs
that you're using a separable kernel (it's not enough just to use a kernel which
happens to be separable).
Similarly, you need to use a RectilinearGrid to specify the grid of inputs -- it's not
enough to ensure that the inputs happen to live on a grid.

In [2]:
using TemporalGPs: Separable, RectilinearGrid

Specify a separable kernel.
The first argument is always the kernel over space, the second the kernel over time.
You can also use weighted sums of separble kernels.

In [3]:
k = Separable(SEKernel(), Matern52Kernel());

Build a GP, and convert it to an SDE as per usual.
Use `ArrayStorage`, not `SArrayStorage`, for these kinds of GPs.

In [4]:
f = to_sde(GP(k), ArrayStorage(Float64));

Construct a rectilinear grid of points in space and time.
Exact inference only works for such grids.
Times must be increasing, points in space can be anywhere, and can be multi-dimensional.
For multi-dimensional spatial-locations, you would probably want to use a ColVecs.

In [5]:
N = 50;
T = 1_000;
points_in_space = collect(range(-3.0, 3.0; length=N));
points_in_time = RegularSpacing(0.0, 0.01, T);
x = RectilinearGrid(points_in_space, points_in_time);

Generate some noisy synthetic data from the GP.

In [6]:
y = rand(f(x, 1e-1));

Construct the posterior as per usual.

In [7]:
f_post = posterior(f(x, 1e-1), y);

Specify some locations at which to make predictions.
This must be another RectilinearGrid with the same spatial locations as x.

In [8]:
T_pr = 1200;
points_in_time_pr = RegularSpacing(0.0, 0.01, T_pr);
x_pr = RectilinearGrid(points_in_space, points_in_time_pr);

Compute the exact posterior marginals at `x_pr`.
This isn't optimised at present, so might take a little while.

In [9]:
f_post_marginals = marginals(f_post(x_pr));
m_post_marginals = mean.(f_post_marginals);
σ_post_marginals = std.(f_post_marginals);

Visualise the posterior marginals. We don't do this during in CI because it causes
problems.

In [10]:
if get(ENV, "TESTING", "FALSE") == "FALSE"
    using Plots
    savefig(
        plot(
            heatmap(reshape(m_post_marginals, N, T_pr)),
            heatmap(reshape(σ_post_marginals, N, T_pr));
            layout=(1, 2),
        ),
        "posterior.png",
    );
end

"/home/runner/work/TemporalGPs.jl/TemporalGPs.jl/docs/src/examples/exact_space_time_inference/posterior.png"

<hr />
<h6>Package and system information</h6>
<details>
<summary>Package information (click to expand)</summary>
<pre>
Status &#96;~/work/TemporalGPs.jl/TemporalGPs.jl/examples/exact_space_time_inference/Project.toml&#96;
  &#91;99985d1d&#93; AbstractGPs v0.5.16
  &#91;98b081ad&#93; Literate v2.14.0
  &#91;91a5bcdd&#93; Plots v1.38.10
  &#91;e155a3c4&#93; TemporalGPs v0.6.3 &#96;/home/runner/work/TemporalGPs.jl/TemporalGPs.jl#61a94af&#96;
</pre>
To reproduce this notebook's package environment, you can
<a href="./Manifest.toml">
download the full Manifest.toml</a>.
</details>
<details>
<summary>System information (click to expand)</summary>
<pre>
Julia Version 1.8.5
Commit 17cfb8e65ea &#40;2023-01-08 06:45 UTC&#41;
Platform Info:
  OS: Linux &#40;x86_64-linux-gnu&#41;
  CPU: 2 × Intel&#40;R&#41; Xeon&#40;R&#41; Platinum 8370C CPU @ 2.80GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 &#40;ORCJIT, icelake-server&#41;
  Threads: 1 on 2 virtual cores
Environment:
  JULIA_DEBUG &#61; Documenter
  JULIA_LOAD_PATH &#61; :/home/runner/.julia/packages/JuliaGPsDocs/e8FS0/src
</pre>
</details>

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*